In [2]:
import socket
import time
import pandas as pd
import os
from bleak import BleakScanner
import asyncio

# Server details
SERVER_HOST = '127.0.0.1'
SERVER_PORT = 5000

# File path for storing Bluetooth device data
EXCEL_FILE_PATH = r'C:\Users\Lenovo\Desktop\bluetooth_devices.xlsx'

# Initialize the Excel file if it doesn't exist
if not os.path.exists(EXCEL_FILE_PATH):
    df = pd.DataFrame(columns=["MAC Address", "Device Name"])
    df.to_excel(EXCEL_FILE_PATH, index=False)

def send_signal_to_csharp(message):
    """Send a message to the C# application."""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        try:
            s.connect((SERVER_HOST, SERVER_PORT))
            s.sendall(message.encode('utf-8'))
            print(f"Message sent to C#: {message}")
        except Exception as e:
            print("Error sending signal to C# application:", e)

def register_or_login_device(mac_address, device_name):
    """Check if the device is new or existing, update the Excel file, and send a signal to C#."""
    # Load the DataFrame, or create it if it doesn’t exist
    if os.path.exists(EXCEL_FILE_PATH):
        df = pd.read_excel(EXCEL_FILE_PATH)
    else:
        df = pd.DataFrame(columns=["MAC Address", "Device Name"])

    # Check if the MAC address already exists
    if mac_address in df["MAC Address"].values:
        print(f"Device {device_name} with MAC {mac_address} is already registered. Logging in.")
        send_signal_to_csharp("User logged in")  # Send "User logged in" message
    else:
        print(f"Registering new device {device_name} with MAC {mac_address}.")
        new_device = pd.DataFrame({"MAC Address": [mac_address], "Device Name": [device_name]})
        df = pd.concat([df, new_device], ignore_index=True)

        # Try saving the Excel file with retry handling
        for attempt in range(3):
            try:
                df.to_excel(EXCEL_FILE_PATH, index=False)
                print(f"New device {device_name} registered successfully.")
                send_signal_to_csharp("User registered")  # Send "User registered" message
                break
            except PermissionError:
                print(f"Attempt {attempt + 1}: Excel file locked. Retrying...")
                time.sleep(1)

async def scan_bluetooth_devices():
    print("Scanning for Bluetooth devices...")
    devices = await BleakScanner.discover()

    for device in devices:
        print(f"Detected Bluetooth device: Name: {device.name}, MAC: {device.address}")
        register_or_login_device(device.address, device.name)

async def main():
    # Continuously scan for devices
    while True:
        await scan_bluetooth_devices()
        await asyncio.sleep(5)  # Control the scan frequency

# Run the main async function
if __name__ == "__main__":
    try:
        asyncio.run(main())
    except RuntimeError as e:
        print("Asyncio event loop issue:", e)
        await main()


Asyncio event loop issue: asyncio.run() cannot be called from a running event loop
Scanning for Bluetooth devices...
Detected Bluetooth device: Name: None, MAC: 18:4A:D5:CC:AC:AF
Device None with MAC 18:4A:D5:CC:AC:AF is already registered. Logging in.
Message sent to C#: User logged in
Detected Bluetooth device: Name: None, MAC: 7C:C9:0F:56:0D:09
Device None with MAC 7C:C9:0F:56:0D:09 is already registered. Logging in.
Message sent to C#: User logged in
Detected Bluetooth device: Name: None, MAC: 15:AC:1A:B0:9E:50
Device None with MAC 15:AC:1A:B0:9E:50 is already registered. Logging in.
Message sent to C#: User logged in
Detected Bluetooth device: Name: [TV] Samsung 7 Series (55), MAC: FC:03:9F:21:1C:3B
Device [TV] Samsung 7 Series (55) with MAC FC:03:9F:21:1C:3B is already registered. Logging in.
Message sent to C#: User logged in
Scanning for Bluetooth devices...
Detected Bluetooth device: Name: [TV] Samsung 7 Series (55), MAC: FC:03:9F:21:1C:3B
Device [TV] Samsung 7 Series (55) wit

CancelledError: 

   ---------------------------------------- 0.0/525.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/525.3 kB ? eta -:--:--
   ---------------------------------------  524.3/525.3 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 525.3/525.3 kB 1.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
